# CarND Project 4, Advance Lane Line
---

## Pipeline, Videos

## 1. Load camera calibration matrix and distortion

In [1]:
from scipy import signal
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt
%matplotlib inline

import pickle

# Load camera calibration matrix and distortion
camera_cal = 'camera_cal/wide_dist_pickle.p'

with open(camera_cal, mode='rb') as f:
    calibration = pickle.load(f) 

mtx = calibration['mtx']
dist = calibration['dist']

## 2. Functions needed to implement Pipeline

In [2]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    if orient == 'x':
        x,y = 1,0
    elif orient == 'y':
        x,y = 0,1
        
    sobel = cv2.Sobel(img, cv2.CV_64F, x, y, ksize=sobel_kernel)
    
    # Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)

    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sbinary

## Sobel Filtering
def sobel_filtering(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    gray =  hls[:,:,2]
        
    # Choose a Sobel kernel size
    ksize = 3 # Choose a larger odd number to smooth gradient measurements

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=ksize, thresh=(10, 100))

    combined = np.zeros_like(gradx)
    combined[(gradx == 1)] = 1
    
    return combined

## HLS Filtering
def hls_filtering(img, channel = 's', thresh = (225, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    if channel == 's':
        ch = hls[:,:,2]
    elif channel == 'l':
        ch = hls[:,:,1]
    elif channel == 'h':
        ch = hls[:,:,0]

    binary = np.zeros_like(ch)
    binary[(ch > thresh[0]) & (ch <= thresh[1])] = 1
    
    return binary

## Perspective Transform
def perspective_transform(img, src, dst):
    img_size = (img.shape[1], img.shape[0])
    
    # Perspective transform 
    M = cv2.getPerspectiveTransform(src, dst)
       
    # Inverse perspective transform
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    perspective = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)  
    return M, Minv, perspective

## Polynomial Fit
def poly_fit(binary_warped, margin):
    
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)

    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 5 #5 actual
    
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Set the width of the windows +/- margin
    #margin = 90  
    
    # Set minimum number of pixels found to recenter window
    minpix = 50
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    

    ## Extra for plotting
    #plt.clf()
    # Generate x and y values for plotting
    #linspace(initial, end, number_of_samples)
    #ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    #left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    #right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    #out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    #out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    #plt.imshow(out_img)
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.xlim(0, 1280)
    #plt.ylim(720, 0)
    
    
    #return left_fit, right_fit, leftx, lefty, rightx, righty, plt
    return left_fit, right_fit, leftx, lefty, rightx, righty

## Radius of Curvature
def radius_curvature(binary_warped, leftx, lefty, rightx, righty):
    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    y_eval = np.max(ploty)

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    x_left = left_fit_cr[0] + left_fit_cr[1]*720*ym_per_pix + left_fit_cr[2]*(720**2)*ym_per_pix
    x_right = right_fit_cr[0] + right_fit_cr[1]*720*ym_per_pix + right_fit_cr[2]*(720**2)*ym_per_pix

    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    # Now our radius of curvature is in meters
    curvature = (left_curverad + right_curverad)/2
    
    return curvature, left_curverad, right_curverad

## Final transformations
def return_image(img, binary_warped, left_fit, right_fit, curvature, Minv):
    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw lane lines
    #color_warp[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 255]
    #color_warp[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [255, 255, 0]


    #Calculate offset position from the center
    #print(pts_left[0,719][0])
    #print(pts_right[0,0][0])
 
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    car_position = ((pts_left[0,719][0]+pts_right[0,0][0])/2)

    offset = (640 - car_position)*xm_per_pix

    if offset > 0:
        position = 'left'
    elif offset < 0:
        position = 'right'
    else:
        position = 'center'

    offset = np.abs(offset)

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)

    #offset, position = car_position(pts_left, pts_right)
    cv2.putText(result, ('Curvature = ' + curvature.astype('str')[0:7]+'m'), (100,100), cv2.FONT_HERSHEY_SIMPLEX, 2, [255,255,255], thickness=2)
    cv2.putText(result, ('Offset = ' + offset.astype('str')[0:3]+'m ' + position), (100,200), cv2.FONT_HERSHEY_SIMPLEX, 2, [255,255,255], thickness=2)

    return result, offset


def check_average(old_average, present_average, present_fit, old_fit, rightx_old, righty_old, rightx, righty, threshold=45):
    check = np.abs(old_average - present_average)
    
    if check>threshold:
        print('old fit')
        return (0.5*old_fit+0.5*present_fit)/2, rightx, righty
    
    return present_fit, rightx, righty

## 3. Pipeline

In [3]:
def lane_lines(img):
    
    global rightx_old, righty_old
    global counter
    global right_fit_old
    
    
    try:
        counter
    except NameError:
        counter = 1
      
    combined = sobel_filtering(img)
    binary = hls_filtering(img)

    thresholded = np.zeros_like(binary)
    thresholded[(combined == 1) | (binary == 1)] = 1
    

    src = np.float32([[727, 450], [1150, 720], [273, 720], [597, 450]])
    dst = np.float32([[1090, 0], [1090, 720], [273, 720], [273, 0]])
    
    M, Minv, perspective = perspective_transform(thresholded, src, dst)
    binary_warped = perspective
    
    if counter < 515:
        margin = 150
    elif counter >= 515 and counter < 1100:
        margin = 90
    elif counter >= 1100:
        margin = 50
        
    left_fit, right_fit, leftx, lefty, rightx, righty = poly_fit(binary_warped, margin)
    
    try:
        right_fit_old
    except NameError:
        right_fit_old = right_fit
        
    #Compare if lane lines separation on present and past frames varies too much     
    x_old = right_fit_old[0]*(500**2) + right_fit_old[1]*500 + right_fit_old[2]
    x = right_fit[0]*(500**2) + right_fit[1]*500 + right_fit[2]
    shift = np.abs(x_old-x)
    
    if shift > 80: #150
        right_fit = right_fit_old
        rightx, righty = rightx_old, righty_old
    

    #plt.savefig('./binary_warped/img_'+str(counter))
    
    try:
        rightx_old
    except NameError:
        rightx_old = rightx
       
    try:
        righty_old
    except NameError:
        righty_old = righty
        
                                        
    curvature, left_curverad, right_curverad = radius_curvature(binary_warped, leftx, lefty, rightx, righty)
    result, offset = return_image(img, binary_warped, left_fit, right_fit, curvature, Minv)
    
    right_fit_old = right_fit
    rightx_old, righty_old = rightx, righty
       
    #cv2.imwrite('./plots/img_'+str(counter)+'.jpg',thresholded)
        
    counter = counter + 1
    
    return result

## 4. Implement Video

In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [5]:
#yellow_output = 'projectvideo_cutted_out.mp4'
#clip2 = VideoFileClip('projectvideo_cutted.mp4')
yellow_output = 'project_video_out.mp4'
clip2 = VideoFileClip('project_video.mp4')
yellow_clip = clip2.fl_image(lane_lines)
%time yellow_clip.write_videofile(yellow_output, audio=False)

#project_video = 'project_video_output.mp4'
#clip1 = VideoFileClip("project_video.mp4")

#project_video_output = clip1.fl_image(lane_lines) #NOTE: this function expects color images!!
#%time project_video_output.write_videofile(project_video, audio=False)
#print(old_poly_left)

[MoviePy] >>>> Building video project_video_out.mp4
[MoviePy] Writing video project_video_out.mp4


100%|█████████▉| 1260/1261 [09:24<00:00,  2.97it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_out.mp4 

CPU times: user 24min 2s, sys: 1min 34s, total: 25min 37s
Wall time: 9min 28s


In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))